---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [69]:
import pandas as pd
import numpy as np

# remove collection_status and compliance_detail
# addresses.csv , latlons.csv, train.csv, test.csv

def blight_model():
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import train_test_split
    
    train_df = pd.read_csv('train.csv',encoding='latin-1').set_index('ticket_id') # readonly/
    test_df = pd.read_csv('test.csv').set_index('ticket_id')
    latlon_df = pd.read_csv('latlons.csv').set_index('address')
    address_df = pd.read_csv('addresses.csv').set_index('ticket_id')
    
    id_latlon_df = address_df.merge(latlon_df,how='left',left_on='address',right_index=True)
    train_df = train_df[~train_df['compliance'].isnull()].merge(id_latlon_df,how='left',left_index=True,right_index=True)
    test_df = test_df.merge(id_latlon_df,how='left',left_index=True,right_index=True)
    train_df['lat'].fillna(method='pad',inplace=True)
    train_df['lon'].fillna(method='pad',inplace=True)
    test_df['lat'].fillna(method='pad',inplace=True)
    test_df['lon'].fillna(method='pad',inplace=True)
    
    X_train = train_df[['judgment_amount','lat','lon']]
    X_test = test_df[['judgment_amount','lat','lon']]
    y_train = train_df['compliance']
    
    

    RF = RandomForestClassifier()
    grid_values = {'n_estimators':[10,20],'min_samples_leaf':[10,50,100]}
    grid_rf_auc = GridSearchCV(RF,param_grid = grid_values, scoring = 'roc_auc')
    grid_rf_auc.fit(X_train,y_train)
    predicted = grid_rf_auc.predict_proba(X_test)[:,1]
    
    return pd.Series(predicted,index=test_df.index)

In [55]:
#blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.105403
285362    0.050738
285361    0.101411
285338    0.128364
285346    0.128364
285345    0.128364
285347    0.128364
285342    0.128364
285530    0.055323
284989    0.064256
285344    0.128364
285343    0.071496
285340    0.069026
285341    0.128364
285349    0.128364
285348    0.128364
284991    0.064256
285532    0.058734
285406    0.041038
285001    0.087012
285006    0.083601
285405    0.050738
285337    0.050738
285496    0.107070
285497    0.107070
285378    0.050738
285589    0.060356
285585    0.099517
285501    0.113206
285581    0.050738
            ...   
376367    0.042365
376366    0.042365
376362    0.050738
376363    0.101411
376365    0.042365
376364    0.042365
376228    0.060987
376265    0.042365
376286    0.157099
376320    0.045165
376314    0.040382
376327    0.305727
376385    0.198715
376435    0.099892
376370    0.305727
376434    0.096804
376459    0.108599
376478    0.037398
376473    0.046273
376484    0.058220
376482    0.055633
37

In [31]:
#train_df.isnull().sum()

ticket_id                          0
agency_name                        0
inspector_name                     0
violator_name                     34
violation_street_number            0
violation_street_name              0
violation_zip_code            250306
mailing_address_str_number      3602
mailing_address_str_name           4
city                               0
state                             93
zip_code                           1
non_us_str_code               250303
country                            0
ticket_issued_date                 0
hearing_date                   12491
violation_code                     0
violation_description              0
disposition                        0
fine_amount                        1
admin_fee                          0
state_fee                          0
late_fee                           0
discount_amount                    0
clean_up_cost                      0
judgment_amount                    0
payment_amount                     0
b

In [32]:
#test_df.isnull().sum()

ticket_id                         0
agency_name                       0
inspector_name                    0
violator_name                    28
violation_street_number           0
violation_street_name             0
violation_zip_code            36977
mailing_address_str_number     1014
mailing_address_str_name          3
city                              1
state                           331
zip_code                          3
non_us_str_code               61001
country                           0
ticket_issued_date                0
hearing_date                   2197
violation_code                    0
violation_description             0
disposition                       0
fine_amount                       0
admin_fee                         0
state_fee                         0
late_fee                          0
discount_amount                   0
clean_up_cost                     0
judgment_amount                   0
grafitti_status               58780
dtype: int64

In [22]:
#address_df.head(5)

,ticket_id,address
0,22056,"2900 tyler, Detroit MI"
1,27586,"4311 central, Detroit MI"
2,22062,"1449 longfellow, Detroit MI"
3,22084,"1441 longfellow, Detroit MI"
4,22093,"2449 churchill, Detroit MI"


In [23]:
#latlon_df.head(5)

,address,lat,lon
0,"4300 rosa parks blvd, Detroit MI 48208",42.346169,-83.079962
1,"14512 sussex, Detroit MI",42.394657,-83.194265
2,"3456 garland, Detroit MI",42.373779,-82.986228
3,"5787 wayburn, Detroit MI",42.403342,-82.957805
4,"5766 haverhill, Detroit MI",42.407255,-82.946295


In [33]:
#train_df = train_df[~train_df['compliance'].isnull()]

In [25]:
#id_lat = address_df.merge(latlon_df,how='left',left_index=True,right_index=True)

In [29]:
#id_lat[id_lat['lat'].isnull()]

,ticket_id,address,lat,lon
65578,89535,"20424 bramford, Detroit MI",NaN,NaN
191722,223598,"445 fordyce, Detroit MI",NaN,NaN
245981,280256,"8300 fordyce, Detroit MI",NaN,NaN
270765,317124,"20424 bramford, Detroit MI",NaN,NaN
278656,329689,"8325 joy rd, Detroit MI 482O4",NaN,NaN
278722,329393,"1201 elijah mccoy dr, Detroit MI 48208",NaN,NaN
282231,333990,"12038 prairie, Detroit MI 482O4",NaN,NaN
305706,367165,"6200 16th st, Detroit MI 482O8",NaN,NaN


In [34]:
#id_lat.set_index('ticket_id',inplace=True)

In [36]:
#id_lat[id_lat['lat'].isnull()]

,address,lat,lon
ticket_id,,,
89535,"20424 bramford, Detroit MI",NaN,NaN
223598,"445 fordyce, Detroit MI",NaN,NaN
280256,"8300 fordyce, Detroit MI",NaN,NaN
317124,"20424 bramford, Detroit MI",NaN,NaN
329689,"8325 joy rd, Detroit MI 482O4",NaN,NaN
329393,"1201 elijah mccoy dr, Detroit MI 48208",NaN,NaN
333990,"12038 prairie, Detroit MI 482O4",NaN,NaN
367165,"6200 16th st, Detroit MI 482O8",NaN,NaN


In [37]:
#train_df.set_index('ticket_id',inplace=True)

In [39]:
#train_df = train_df.merge(id_lat,how='left',left_index=True,right_index=True)

In [41]:
#train_df[train_df['lat'].isnull()]

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
89535,Department of Public Works,"Stanford, Elton","CHASE BANK, JP MORGAN",20424.0,BRAMFORD,NaN,127.0,NORTHLAND DRIVE,MENDOTA HEIGHTS,MN,...,85.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"20424 bramford, Detroit MI",NaN,NaN
223598,"Buildings, Safety Engineering & Env Department","O'Neal, Claude","HJADJANTONI, MICHAEL",445.0,FORDYCE,NaN,437.0,FOREST,DETROIT,MI,...,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0,"445 fordyce, Detroit MI",NaN,NaN


In [42]:
#test_df.set_index('ticket_id',inplace=True)

In [44]:
#test_df = test_df.merge(id_lat,how='left',left_index=True,right_index=True)

In [45]:
#test_df[test_df['lat'].isnull()]

,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,state,...,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,grafitti_status,address,lat,lon
ticket_id,,,,,,,,,,,,,,,,,,,,,
317124,Detroit Police Department,"Nichols, Douglas","cherry, christopher",20424.0,BRAMFORD,NaN,17125,w.14 mile rd,beverly hills,MI,...,20.0,10.0,50.0,0.0,0.0,580.0,NaN,"20424 bramford, Detroit MI",NaN,NaN
329689,Department of Public Works,"Dean, Jered","GREEN, NIKKA N",8325.0,JOY RD,482O4,26638,CARNEGIE PARK,SOUTHFIELD,MI,...,20.0,10.0,10.0,0.0,0.0,140.0,NaN,"8325 joy rd, Detroit MI 482O4",NaN,NaN
329393,Department of Public Works,"Zizi, Josue",RESEARCH PARK DETROIT LIMITED DIVIDEND HOUSING,1201.0,ELIJAH MCCOY DR,48208,12,SOUTH MAIN ST,MOUNT PLEASANT,MI,...,20.0,10.0,25.0,0.0,0.0,305.0,NaN,"1201 elijah mccoy dr, Detroit MI 48208",NaN,NaN
333990,Department of Public Works,"McClain, Melvin","JACKSON, LASANDRA",12038.0,PRAIRIE,482O4,7137,TUXEDO,DETROIT,MI,...,20.0,10.0,50.0,0.0,0.0,580.0,NaN,"12038 prairie, Detroit MI 482O4",NaN,NaN
367165,Department of Public Works,"Williamson, Lillett",CLOSED LOOP ECONOMMY GROUP,6200.0,16TH ST,482O8,487,ALEXANDRINE,DETROIT,MI,...,20.0,10.0,0.0,0.0,0.0,130.0,NaN,"6200 16th st, Detroit MI 482O8",NaN,NaN


In [62]:
#train_df['violation_code'].head(5)

ticket_id
22056     9-1-36(a)
27586    61-63.0600
22046     9-1-36(a)
18738    61-63.0500
18735    61-63.0100
Name: violation_code, dtype: object